In [1]:
import pandas as pd
import pyrankvote
from pathlib import Path

In [2]:
df = pd.read_csv(Path('./data/20200629.csv').resolve())
df

,Timestamp,Rank your movie choices. You don't have to do a full ranking. [First Choice],Rank your movie choices. You don't have to do a full ranking. [Second Choice],Rank your movie choices. You don't have to do a full ranking. [Third Choice],Rank your movie choices. You don't have to do a full ranking. [Fourth Choice],Rank your movie choices. You don't have to do a full ranking. [Fifth Choice],Rank your movie choices. You don't have to do a full ranking. [Sixth Choice],What theme should we explore next week? [First Choice],What theme should we explore next week? [Second Choice],What theme should we explore next week? [Third Choice],What theme should we explore next week? [Fourth Choice],What theme should we explore next week? [Fifth Choice],What theme should we explore next week? [Sixth Choice],What theme should we explore next week? [Seventh Choice]
0,6/25/2020 20:01:08,End of Days,Faust,Angel Heart,Häxan,Simon of the Desert,Petey Wheatstraw,Heist Movie,Set on an island,No Theme/Free-for-all,Steven Seagal,Woman-led Westerns,TEENS!,Budget under $1 million
1,6/25/2020 20:05:55,End of Days,Faust,Angel Heart,Petey Wheatstraw,Häxan,Simon of the Desert,Steven Seagal,Heist Movie,Woman-led Westerns,No Theme/Free-for-all,TEENS!,Set on an island,Budget under $1 million
2,6/25/2020 20:20:04,Faust,Simon of the Desert,Häxan,Angel Heart,Petey Wheatstraw,End of Days,No Theme/Free-for-all,Woman-led Westerns,Set on an island,TEENS!,Heist Movie,Budget under $1 million,Steven Seagal
3,6/25/2020 20:28:10,Petey Wheatstraw,Angel Heart,Faust,NaN,NaN,NaN,TEENS!,Set on an island,Budget under $1 million,NaN,NaN,NaN,NaN
4,6/25/2020 20:55:12,Faust,Häxan,Simon of the Desert,Angel Heart,Petey Wheatstraw,End of Days,Budget under $1 million,Heist Movie,Set on an island,TEENS!,Woman-led Westerns,Steven Seagal,No Theme/Free-for-all
5,6/25/2020 21:11:17,Häxan,Faust,Simon of the Desert,Angel Heart,Petey Wheatstraw,End of Days,TEENS!,Heist Movie,Budget under $1 million,Steven Seagal,NaN,NaN,NaN
6,6/26/2020 7:20:06,Angel Heart,Petey Wheatstraw,Simon of the Desert,Faust,NaN,NaN,Budget under $1 million,Woman-led Westerns,Heist Movie,TEENS!,Set on an island,Steven Seagal,No Theme/Free-for-all
7,6/26/2020 8:45:28,End of Days,Simon of the Desert,Petey Wheatstraw,Angel Heart,Häxan,Faust,Budget under $1 million,TEENS!,Set on an island,Heist Movie,Steven Seagal,Woman-led Westerns,No Theme/Free-for-all
8,6/26/2020 9:52:15,Angel Heart,Petey Wheatstraw,Faust,End of Days,Simon of the Desert,Häxan,Woman-led Westerns,Set on an island,TEENS!,Heist Movie,Budget under $1 million,Steven Seagal,No Theme/Free-for-all
9,6/27/2020 18:00:08,Simon of the Desert,Faust,Häxan,NaN,NaN,NaN,TEENS!,Budget under $1 million,Set on an island,NaN,NaN,NaN,NaN


In [3]:
just_ballots = df.loc[:, df.columns.str.startswith('Rank')]
next_week = df.loc[:, df.columns.str.startswith('What')]

In [4]:
submissions = just_ballots.loc[just_ballots.notna().all(axis=1).idxmax(),:].values
submissions

array(['End of Days', 'Faust', 'Angel Heart', 'Häxan',
       'Simon of the Desert', 'Petey Wheatstraw'], dtype=object)

In [5]:
submission_dict = {submission: pyrankvote.Candidate(submission) for submission in submissions}

In [6]:
balloted = just_ballots.applymap(lambda x: submission_dict[x] if isinstance(x, str) else x)

In [7]:
submission_dict

{'End of Days': <Candidate('End of Days')>,
 'Faust': <Candidate('Faust')>,
 'Angel Heart': <Candidate('Angel Heart')>,
 'Häxan': <Candidate('Häxan')>,
 'Simon of the Desert': <Candidate('Simon of the Desert')>,
 'Petey Wheatstraw': <Candidate('Petey Wheatstraw')>}

In [8]:
ballots = []
for ballot in balloted.itertuples():
    ballots.append(pyrankvote.Ballot(ranked_candidates=[b for b in ballot if isinstance(b, pyrankvote.Candidate)]))

In [9]:
ballots

[<Ballot(End of Days, Faust, Angel Heart, Häxan, Simon of the Desert, Petey Wheatstraw)>,
 <Ballot(End of Days, Faust, Angel Heart, Petey Wheatstraw, Häxan, Simon of the Desert)>,
 <Ballot(Faust, Simon of the Desert, Häxan, Angel Heart, Petey Wheatstraw, End of Days)>,
 <Ballot(Petey Wheatstraw, Angel Heart, Faust)>,
 <Ballot(Faust, Häxan, Simon of the Desert, Angel Heart, Petey Wheatstraw, End of Days)>,
 <Ballot(Häxan, Faust, Simon of the Desert, Angel Heart, Petey Wheatstraw, End of Days)>,
 <Ballot(Angel Heart, Petey Wheatstraw, Simon of the Desert, Faust)>,
 <Ballot(End of Days, Simon of the Desert, Petey Wheatstraw, Angel Heart, Häxan, Faust)>,
 <Ballot(Angel Heart, Petey Wheatstraw, Faust, End of Days, Simon of the Desert, Häxan)>,
 <Ballot(Simon of the Desert, Faust, Häxan)>,
 <Ballot(End of Days, Angel Heart, Simon of the Desert)>,
 <Ballot(End of Days)>]

In [10]:
election_result = pyrankvote.instant_runoff_voting(submission_dict.values(), ballots,)

In [11]:
print(election_result)

ROUND 1
Candidate              Votes  Status
-------------------  -------  --------
End of Days                5  Hopeful
Faust                      2  Hopeful
Angel Heart                2  Hopeful
Simon of the Desert        1  Hopeful
Petey Wheatstraw           1  Hopeful
Häxan                      1  Rejected

ROUND 2
Candidate              Votes  Status
-------------------  -------  --------
End of Days                5  Hopeful
Faust                      3  Hopeful
Angel Heart                2  Hopeful
Simon of the Desert        1  Rejected
Petey Wheatstraw           1  Rejected
Häxan                      0  Rejected

ROUND 3
Candidate              Votes  Status
-------------------  -------  --------
End of Days                5  Hopeful
Faust                      4  Hopeful
Angel Heart                3  Rejected
Simon of the Desert        0  Rejected
Petey Wheatstraw           0  Rejected
Häxan                      0  Rejected

FINAL RESULT
Candidate              Votes  Status
---

In [12]:
submissions = next_week.loc[next_week.notna().all(axis=1).idxmax(),:].values
submission_dict = {submission: pyrankvote.Candidate(submission) for submission in submissions}
balloted = next_week.applymap(lambda x: submission_dict[x] if isinstance(x, str) else x)
ballots = []
for ballot in balloted.itertuples():
    ballots.append(pyrankvote.Ballot(ranked_candidates=[b for b in ballot if isinstance(b, pyrankvote.Candidate)]))
print(pyrankvote.instant_runoff_voting(submission_dict.values(), ballots,))

ROUND 1
Candidate                  Votes  Status
-----------------------  -------  --------
TEENS!                         5  Hopeful
Budget under $1 million        3  Hopeful
Heist Movie                    1  Hopeful
Woman-led Westerns             1  Hopeful
No Theme/Free-for-all          1  Hopeful
Steven Seagal                  1  Rejected
Set on an island               0  Rejected

ROUND 2
Candidate                  Votes  Status
-----------------------  -------  --------
TEENS!                         5  Hopeful
Budget under $1 million        3  Hopeful
Heist Movie                    2  Hopeful
Woman-led Westerns             1  Rejected
No Theme/Free-for-all          1  Rejected
Steven Seagal                  0  Rejected
Set on an island               0  Rejected

FINAL RESULT
Candidate                  Votes  Status
-----------------------  -------  --------
TEENS!                         7  Elected
Budget under $1 million        3  Rejected
Heist Movie                    2  Reje